In [1]:
#CARREGA A(S) BIBLIOTECAS
import pandas as pd
import json

In [2]:
#CARREGA O ARQUIVO QUE CONTEM A LOCALIZAÇÃO DAS PLANTAS DE PRODUCAO DE ÓLEO LUBRIFICANTE EM SOLO BRASILEIRO
plantas_producao = pd.read_excel("../data/boletim_1/Anexo_E.xlsx", sheetname="30-Mapa_Planta_Prod_OLAC")
plantas_producao.head()

,CNPJ,Produtor,Município,UF
0,57941890000153,AGECOM PRODUTOS DE PETRÓLEO LTDA.,MAUA,SP
1,3439153000128,BIOLUB QUÍMICA LTDA -ME,SOROCABA,SP
2,33194978000190,CASTROL BRASIL LTDA,RIO DE JANEIRO,RJ
3,45036670000104,CHEMLUB PRODUTOS QUÍMICOS LTDA,VALINHOS,SP
4,5524572001084,CHEVRON BRASIL LUBRIFICANTES LTDA.,DUQUE DE CAXIAS,RJ


In [3]:
#CRIA UM DICIONÁRIO PARA ARMAZENAR O UF DAS PLANTAS DE PRODUCAO
empresa_prod_uf = {}

for index,row in plantas_producao.iterrows():
    empresa = row['Produtor']
    if empresa not in empresa_prod_uf:
        empresa_prod_uf[empresa] = row['UF']
    else:
    #CASO UMA EMPRESA TENHA MAIS DE UMA PLANTA INDUSTRIAL, IMPRIMIR O NOME DA EMPRESA E A UF ONDE A PLANTA ESTÁ.    
        print(empresa+" - "+row['UF']) # A IPIERANGA NESSE CASO POSSUI DUAS PLANTAS NO MESMO ESTADO (RJ)
        continue

IPIRANGA PRODUTOS DE PETRÓLEO S.A - RJ


In [4]:
#Vendas nacionais de OLAC pelos Produtores e Importadores
#A COLUNA PRODUTOR-IMPORTADOR POSSUI A INFORMAÇÃÕ DE UMA EMPRESA QUE DECLARA UMA OPERAÇÃO DE COMERCIALIZAÇÃO À ANP.
#TODAVIA, ESSE PRODUTO COMERCIALIZADO PODE TER A SEGUINTE ORIGEM:
### 1. SER PRODUZIDO EM PLANTA INDUSTRIAL PRÓPRIA EM SOLO BRASILEIRO.
### 2. SER PRODUZIDO EM PLANTA INDUSTRIAL DE TERCEIRO EM SOLO BRASILEIRO.
### 3. SER PRODUZID FORA DO BRASIL (PLANTA PRÓPRIA OU NÃO.)
vendas = pd.read_excel("../data/boletim_1/Anexo_H.xlsx", sheetname="41-Vendas OLAC Geral")
vendas.head(5)

,Data,Cód Regulado ANP,Produtor-Importador,Código do Produto,Descrição do Produto,Região do Destinatário,UF do Destinatário,Volume-Litros
0,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,PR,9828
1,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,SC,800
2,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,PR,3685
3,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,RS,480
4,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,SC,5040


In [5]:
#TROCAR O NOME DA EMPRESA (PRODUTOR-IMPORTADOR) PELO UF ONDE ESTÁ LOCALIZADO A PLANTA DE PRODUÇÃO.
vendas['Produtor-Importador'] = vendas['Produtor-Importador'].map(empresa_prod_uf)

In [6]:
#NÚMERO DE REGISTROS DE DECLARAÇÃO DE COMERCIALIZAÇÃO DE EMPRESAS QUE NÃO PRODUZIRAM EM PLANTA PRÓPRIA NO BRASIL OU SÃO 
#CONSIDERADAS IMPORTADORAS
vendas['Produtor-Importador'].isnull().sum()

9848

In [7]:
#PERCENTUAL DE REGISTROS DE DECLARAÇÃO DE COMERCIALIZAÇÃO QUE SÃO DE EMPRESAS QUE NÃO PRODUZIRAM EM PLANTA PRÓPRIA NO BRASIL OU SÃO 
#CONSIDERADAS IMPORTADORAS
str(round(vendas['Produtor-Importador'].isnull().sum()/vendas['Produtor-Importador'].shape[0]*100,2))+"%"

'26.14%'

In [8]:
#SOMA DOS VOLUMES DE COMERCIALIZACAO DE EMPRESAS DETENTORAS & PRODUTORAS
count_volume_detentor_produtor = 0

#SOMA DOS VOLUMES DE EMPRESAS QUE SÃO DETENTORAS/TERCERIZADORAS OU DETENTORAS/IMPORTADORAS
count_volume_outros = 0
for index,row in vendas.iterrows():
    if type(row['Produtor-Importador']) != str:
        count_volume_outros += row['Volume-Litros']
    else:
        count_volume_detentor_produtor += row['Volume-Litros']

In [9]:
print("Volume total de empresas Detentoras/Produtoras "+str(count_volume_detentor_produtor/1000)+ " m³")
print("Volume total de empresas Detentoras/Tercerizadoras/Importadoras "+str(count_volume_outros/1000)+ " m³")

Volume total de empresas Detentoras/Produtoras 1131775.986 m³
Volume total de empresas Detentoras/Tercerizadoras/Importadoras 97916.397 m³


In [10]:
#PERCENTUAL EM VOLUME DOS REGISTROS QUE NÃO PODE SE INFERIR QUAL A ORIGEM DA PRODUÇÃO, APENAS O DESTINATÁRIO.
str(round(count_volume_outros/(count_volume_outros+count_volume_detentor_produtor)*100,2))+"%"

'7.96%'

In [11]:
#LISTA PARA ABRIGAR O INDEX DOS REGISTROS QUE NÃO POSSUEM A IDENTIFICAÇÃO DA UF DA ORIGEM DA PRODUÇÃO
list_index_nan = []
for index,row in vendas.iterrows():
    if type(row['Produtor-Importador']) != str:
        list_index_nan.append(index)

In [12]:
#DELETAR DO DATAFRAME DOS OS REGISTROS QUE NÃO POSSUEM A IDENTIFICAÇÃO DA UF DA ORIGEM DA PRODUÇÃO
vendas_uf = vendas[vendas['Produtor-Importador'].notnull()]

In [13]:
#RESETAR O INDEX DO DATAFRAME
vendas_uf.reset_index(inplace=True)
del vendas_uf['index']

In [14]:
vendas_uf.columns = ['Data', 'Cód Regulado ANP', 'UF_origem',
       'Código do Produto', 'Descrição do Produto', 'Região do Destinatário',
       'UF_destino', 'Volume-Litros']

In [15]:
#CRIA UMA LISTA COM TODAS AS UF'S
lista_UF = vendas_uf['UF_destino'].unique().tolist()

#ORGANIZA A LISTA POR ORDEM ALFABÉTICA
lista_UF.sort()

In [16]:
#LISTA DE DICIONÁRIOS
lista_dict_uf = []
for uf in lista_UF:
    dict_uf = {"name": uf}
    lista_dict_uf.append(dict_uf)

In [17]:
#CRIA UM DICIONÁRIO PARA CODIFICAR AS UF'S
uf_codigo = {}
count = 0

for uf in lista_UF:
    uf_codigo[uf] = count
    count += 1

In [18]:
#CODIFICA AS COLUNAS UTILIZANDO O DICIONÁRIO uf_codigo
vendas_uf['UF_origem'] = vendas_uf['UF_origem'].map(uf_codigo)
vendas_uf['UF_destino'] = vendas_uf['UF_destino'].map(uf_codigo)

C:\Users\jferraz\AppData\Local\Continuum\Anaconda3\envs\pml\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\jferraz\AppData\Local\Continuum\Anaconda3\envs\pml\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [19]:
#VERIFICACÃO DA CODIFICACAO
vendas_uf.head()

,Data,Cód Regulado ANP,UF_origem,Código do Produto,Descrição do Produto,Região do Destinatário,UF_destino,Volume-Litros
0,12/2016,200296,17,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,17,9828
1,12/2016,200296,17,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,23,800
2,12/2016,200296,17,620501001,Ciclo Otto,SUL,17,3685
3,12/2016,200296,17,620501001,Ciclo Otto,SUL,22,480
4,12/2016,200296,17,620501001,Ciclo Otto,SUL,23,5040


In [20]:
#CRIAR UMA LISTA DE DICIONÁRIOS PARA RECEBER OS DADOS DE LIGAÇÃO ENTRE OS NÓS
lista_dict_links = []
for index,row in vendas_uf.iterrows():
    dict_links = {"source": row['UF_origem'], "target": row['UF_destino'], "value": row['Volume-Litros']}
    lista_dict_links.append(dict_links)

In [21]:
#CRIAR UM OBJETO PARA CRIAR O DIAGRAMA DE SANKEY
data_vendas_olac = { "nodes" : lista_dict_uf , "links" : lista_dict_links }

In [23]:
data_vendas_olac

{'links': [{'source': 17, 'target': 17, 'value': 9828},
  {'source': 17, 'target': 23, 'value': 800},
  {'source': 17, 'target': 17, 'value': 3685},
  {'source': 17, 'target': 22, 'value': 480},
  {'source': 17, 'target': 23, 'value': 5040},
  {'source': 17, 'target': 17, 'value': 1184},
  {'source': 17, 'target': 17, 'value': 17764},
  {'source': 17, 'target': 22, 'value': 13800},
  {'source': 17, 'target': 23, 'value': 71772},
  {'source': 17, 'target': 17, 'value': 30519},
  {'source': 17, 'target': 23, 'value': 2200},
  {'source': 25, 'target': 25, 'value': 7220},
  {'source': 25, 'target': 25, 'value': 117400},
  {'source': 25, 'target': 8, 'value': 180000},
  {'source': 25, 'target': 25, 'value': 15840},
  {'source': 17, 'target': 6, 'value': 29280},
  {'source': 17, 'target': 8, 'value': 15720},
  {'source': 17, 'target': 4, 'value': 9600},
  {'source': 17, 'target': 5, 'value': 4560},
  {'source': 17, 'target': 9, 'value': 1920},
  {'source': 17, 'target': 14, 'value': 13160},


In [38]:
#CRIA UMA LISTA COM A IDENTIFICACAO DOS ESTADOS DE ORIGEM E DE DESTINO
lista_source = []
lista_target = []
for data in data_vendas_olac['links']:
    if data['source'] not in lista_source:
        lista_source.append(data['source'])
    if data['target'] not in lista_target:
        lista_target.append(data['target'])


In [87]:
#CRIA UM DICIONÁRIO ONDE IRÁ RECEBER A INFORMAÇÃO AGREGADA DAS MOVIMENTAÇÕES.
#O OBJETIVO É TER UM REGISTRO ÚNICO DE MOVIMENTAÇÃO (SOMÁTORIO) ENTRE DUAS ENTIDADES
nome_dict = []
for UFcod in lista_source:
    name = UFcod
    nome_dict.append(name)
dict_list_source={ name:{} for name in nome_dict}

In [89]:
#ALIMENTA O DICIONÁRIO COM OS DADOS AGREGADOS
for data in data_vendas_olac['links']:
    for source in dict_list_source:
        if data['source'] == source:
            if data['target'] not in dict_list_source[source]:
                dict_list_source[source][data['target']] = data['value']
            else:
                dict_list_source[source][data['target']] += data['value']   
    

In [91]:
#VISUALIZAR A ESTRUTURA DO DICIONÁRIO CRIADO
dict_list_source

{2: {0: 44217, 2: 610813, 13: 4192, 20: 520438, 21: 224457},
 4: {1: 288,
  2: 360,
  4: 1576303,
  5: 231547,
  7: 120,
  8: 416,
  10: 63173,
  11: 1490,
  13: 240,
  14: 1060,
  16: 307622,
  17: 292,
  18: 4800,
  19: 114900},
 10: {0: 287527,
  1: 857948,
  2: 863416,
  3: 114704,
  4: 5082052,
  5: 1933558,
  6: 2022515,
  7: 2755753,
  8: 4616858,
  9: 1987397,
  10: 26391539,
  11: 2829555,
  12: 5589156,
  13: 2862358,
  14: 1178812,
  15: 3279734,
  16: 970156,
  17: 8892039,
  18: 5355512,
  19: 1281716,
  20: 867851,
  21: 220852,
  22: 6174731,
  23: 4896386,
  24: 499352,
  25: 25148474,
  26: 816056},
 12: {8: 1000, 11: 327224, 12: 389336, 13: 57892, 20: 129600, 25: 155388},
 17: {0: 696,
  1: 11672,
  2: 154988,
  3: 89464,
  4: 486886,
  5: 317768,
  6: 248125,
  7: 107108,
  8: 327080,
  9: 313160,
  10: 423598,
  11: 213166,
  12: 568844,
  13: 180298,
  14: 181692,
  15: 439738,
  16: 103908,
  17: 3511061,
  18: 192024,
  19: 245188,
  20: 373134,
  21: 13360,
  22

In [130]:
#IMPRESSÃO DOS DADOS AGREGADOS
for key,values in dict_list_source.items():
    for val in values.items():
        print('source: '+str(key)+" => "+"target: "+str(val[0])+" => "+"value: "+str(val[1]))

source: 17 => target: 17 => value: 3511061
source: 17 => target: 23 => value: 2077832
source: 17 => target: 22 => value: 974880
source: 17 => target: 6 => value: 248125
source: 17 => target: 8 => value: 327080
source: 17 => target: 4 => value: 486886
source: 17 => target: 5 => value: 317768
source: 17 => target: 9 => value: 313160
source: 17 => target: 14 => value: 181692
source: 17 => target: 15 => value: 439738
source: 17 => target: 16 => value: 103908
source: 17 => target: 19 => value: 245188
source: 17 => target: 24 => value: 50488
source: 17 => target: 25 => value: 357828
source: 17 => target: 20 => value: 373134
source: 17 => target: 11 => value: 213166
source: 17 => target: 12 => value: 568844
source: 17 => target: 2 => value: 154988
source: 17 => target: 26 => value: 25596
source: 17 => target: 7 => value: 107108
source: 17 => target: 10 => value: 423598
source: 17 => target: 18 => value: 192024
source: 17 => target: 1 => value: 11672
source: 17 => target: 3 => value: 89464
sou

In [131]:
#CRIAR UMA LISTA DE DICIONÁRIOS PARA RECEBER OS DADOS AGREGADOS DE LIGAÇÃO ENTRE OS NÓS
lista_dict_agg = []
for key,values in dict_list_source.items():
    for val in values.items():
        dict_links = {"source": key, "target": val[0], "value": val[1]}
        lista_dict_agg.append(dict_links)

In [132]:
lista_dict_agg

[{'source': 17, 'target': 17, 'value': 3511061},
 {'source': 17, 'target': 23, 'value': 2077832},
 {'source': 17, 'target': 22, 'value': 974880},
 {'source': 17, 'target': 6, 'value': 248125},
 {'source': 17, 'target': 8, 'value': 327080},
 {'source': 17, 'target': 4, 'value': 486886},
 {'source': 17, 'target': 5, 'value': 317768},
 {'source': 17, 'target': 9, 'value': 313160},
 {'source': 17, 'target': 14, 'value': 181692},
 {'source': 17, 'target': 15, 'value': 439738},
 {'source': 17, 'target': 16, 'value': 103908},
 {'source': 17, 'target': 19, 'value': 245188},
 {'source': 17, 'target': 24, 'value': 50488},
 {'source': 17, 'target': 25, 'value': 357828},
 {'source': 17, 'target': 20, 'value': 373134},
 {'source': 17, 'target': 11, 'value': 213166},
 {'source': 17, 'target': 12, 'value': 568844},
 {'source': 17, 'target': 2, 'value': 154988},
 {'source': 17, 'target': 26, 'value': 25596},
 {'source': 17, 'target': 7, 'value': 107108},
 {'source': 17, 'target': 10, 'value': 423598},

In [133]:
#CRIAR UM OBJETO PARA CRIAR O DIAGRAMA DE SANKEY COM DADOS AGREGADOS
vendas_olac_agg = { "nodes" : lista_dict_uf , "links" : lista_dict_agg }

In [134]:
vendas_olac_agg 

{'links': [{'source': 17, 'target': 17, 'value': 3511061},
  {'source': 17, 'target': 23, 'value': 2077832},
  {'source': 17, 'target': 22, 'value': 974880},
  {'source': 17, 'target': 6, 'value': 248125},
  {'source': 17, 'target': 8, 'value': 327080},
  {'source': 17, 'target': 4, 'value': 486886},
  {'source': 17, 'target': 5, 'value': 317768},
  {'source': 17, 'target': 9, 'value': 313160},
  {'source': 17, 'target': 14, 'value': 181692},
  {'source': 17, 'target': 15, 'value': 439738},
  {'source': 17, 'target': 16, 'value': 103908},
  {'source': 17, 'target': 19, 'value': 245188},
  {'source': 17, 'target': 24, 'value': 50488},
  {'source': 17, 'target': 25, 'value': 357828},
  {'source': 17, 'target': 20, 'value': 373134},
  {'source': 17, 'target': 11, 'value': 213166},
  {'source': 17, 'target': 12, 'value': 568844},
  {'source': 17, 'target': 2, 'value': 154988},
  {'source': 17, 'target': 26, 'value': 25596},
  {'source': 17, 'target': 7, 'value': 107108},
  {'source': 17, '

In [22]:
with open('../data/vendas_olac_uf.json', 'w') as fp:
    json.dump(data_vendas_olac, fp)

In [135]:
with open('../data/vendas_olac_agg.json', 'w') as fp:
    json.dump(vendas_olac_agg, fp)